In [ ]:
# Setup scenario
import geopandas as gpd
import numpy as np
import pandas as pd

from climada.engine import CostBenefit, Impact, ImpactCalc
from climada.engine.cost_benefit import risk_aai_agg
from climada.entity import DiscRates, Entity, Exposures, LitPop
from climada.entity.impact_funcs import ImpactFunc, ImpactFuncSet, ImpfTropCyclone
from climada.entity.impact_funcs.storm_europe import ImpfStormEurope
from climada.entity.impact_funcs.trop_cyclone import ImpfSetTropCyclone
from climada.entity.measures import Measure, MeasureSet
from climada.hazard import Hazard
from climada.util.api_client import Client

import handlers

from logger_config import LoggerConfig

logger = LoggerConfig(logger_types=["file"])


country_name = "Egypt"          # Available countries: ['Egypt', 'Thailand']
exposure_type = "litpop"        # Available exposure types for Egypt/Thailand: ['litpop']
hazard_type = "river_flood"     # Available hazard types for Egypt/Thailand: ['river_flood', 'wildfire', 'earthquake', 'flood', 'tropical_cyclone']
scenario = "historical"         # Available scenarios for Egypt/Thailand: ['rcp26', 'historical', 'rcp60', 'rcp85']
time_horizon = "1980_2000"      # Available time horizons for Egypt/Thailand: ['2030_2050', '1980_2000', '2070_2090', '2010_2030', '2050_2070'] / 
annual_growth = 1.02

client = Client()

In [ ]:
exposure_present = handlers.get_exposure(country_name)

In [ ]:
hazard_present = handlers.get_hazard(hazard_type, scenario, time_horizon, country_name)

In [ ]:
impact_function_set = handlers.calculate_impact_function_set(hazard_present)
impact_present = handlers.calculate_impact(exposure_present, hazard_present, impact_function_set)

In [ ]:
adaptation_measures = {
    "name": "Measure 1",
    "haz_type": "RF",
    "cost": 100000,
    "hazard_inten_imp": (1, -1),
}

measure_list = []

def get_measure(adaptation_measures: dict = None) -> Measure:
    if adaptation_measures is None:
        return Measure()
    return Measure(**adaptation_measures)

measure1 = get_measure(adaptation_measures)
measure_list.append(measure1)

measure_set = MeasureSet(measure_list)
measure_set.check()

In [ ]:
years=np.arange(exposure_present.ref_year, 2040)
rates=np.ones(years.size) * 0.02
# rates=np.zeros(years.size)
discount_rates = DiscRates(years=years, rates=rates)
discount_rates.check()
# discount_rates.plot()

In [ ]:
# Create Entity objects
entity_present = Entity(
    exposures=exposure_present,
    disc_rates=discount_rates,
    impact_func_set=impact_function_set,
    measure_set=measure_set,
)

In [ ]:
# Calculate cost-benefit
cost_benefit = CostBenefit()
cost_benefit.calc(hazard_present, entity_present)
cost_benefit.plot_cost_benefit()
cost_benefit.plot_event_view((25,))

In [ ]:
# define this as a function because we'll use it again later
def waterfall():
  return costben_measures_only.plot_waterfall(hazard_present, entity_present, hazard_future, entity_future,
                                risk_func=risk_aai_agg)

ax = waterfall()

In [ ]:
costben_measures_only.plot_arrow_averted(axis = waterfall(), in_meas_names=['Measure A', 'Measure B'], accumulate=True, combine=False,
                           risk_func=risk_aai_agg, disc_rates=None, imp_time_depen=1)

In [ ]:
impact_function_set.plot()

In [ ]:
entity_present = Entity().from_excel(ENTITIES_DIR / "entity_template.xlsx")
entity_present.check()
entity_present.measures.get_measure('TC')[0].name